In [1]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/
!git clone https://github.com/oglee815/mygpt-lecture.git
%cd /content/drive/MyDrive/mygpt-lecture

/content/drive/MyDrive
Cloning into 'mygpt-lecture'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 190 (delta 39), reused 160 (delta 26), pack-reused 0
Receiving objects: 100% (190/190), 5.15 MiB | 6.95 MiB/s, done.
Resolving deltas: 100% (39/39), done.
Updating files: 100% (136/136), done.
/content/drive/MyDrive/mygpt-lecture


# Stage 1, 도메인 특화 사전학습 데이터를 만들어보자

### Domain Adaptive Pretraining 데이터 만들기 (100% 수작업)

In [5]:
data=[
    {'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 개요1\n',
    'output':"국선도(밝돌법)는 9,800년의 역사와 전통을 가진 것으로 알려진 한국 고유의 심신 수련법으로, 남녀노소, 인종, 종교, 질병 유무와 무관하게 누구든지 심신 수련을 통해 건강한 몸, 마음과 정신을 가진 전인적인 인간으로 거듭날 수 있도록 돕는 자기계발 수련법이다.\
    9단법 37단계 430여 개 동작으로 구성된 점진적이고 체계적인 단전행공(丹田行功)의 도법(道法)으로 한국 고유의 명상과학 프로그램이다.\
    심신수련과 인재 양성의 도법으로 약 9,800년 전부터 전해져 내려와 고조선의 소도, 고구려의 조의선인제도, 신라의 화랑도 등으로 불렸고, 고려 이후에는 산중 도법으로 전해 내려오다가 산중 수도인 청산(靑山, 본명 고한영)이 스승 청운도인의 \"모두의 유익을 위해 도법을 전하라\"는 명을 받고 1967년에 사회로 나와 국선도(밝돌법)를 현대 인류 사회에 다시 소개하였다.\
    밝돌법은 국선도의 본래 이름으로, ‘밝’은 생명의 에너지(태양, 밝음), ‘돌’은 순환하는(돌고 도는) 우주 자연의 법칙을 뜻하며, ‘밝돌법’은 우주 질서의 순리에 맞게 생명을 양생(養生)하는 수련방법을 의미한다.\
    국선도는 상고시대부터 전해오는 한국의 고유한 전통 심신수련법이다."
    },
    {'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 개요2\n',
    'output':"국선도는 상고시대부터 전해오는 한국의 고유한 전통 심신수련법이다.\
    청산선사로 알려진 수도인 고한영이 청운도인 (본명 이송운, 경상북도 안동 출신)으로부터 전수한 법통으로, 동방 단리(丹理)의 정통적 정수로 정기신(精氣神) 삼단(三段) 단전(丹田) 이단(二段) 호흡으로부터 대기와 합일시키는 경지까지 승화시키는 묘법으로 알려져 있다.\
    삼국시대 이전까지만 하더라도 사람들 사이에서 널리 행해진 것으로 많은 문헌을 통해 알려져 있다. 삼국시대 이후로는 깊은 산중에서만 비전되어 오다가 무운도인의 대를 이은 청운도인으로부터 청산선사가 그 정통 수련법을 전수 받아 1967년 하산하여 전파하면서 다시 세상에 알려지게 되었다.\
    유구한 역사를 통해 이어져 내려 오는 배달민족의 혼과 민족정기가 담긴 한국 고유의 수련법으로 원래 명칭은 밝돌법(밝돌: 배달)으로 알려져 있다.\
    국선도는 수천 년의 체험을 통해 자연의 생성원리와 인체의 생리구조에 맞게 고안된 수백 개의 몸 동작과 정교한 호흡법이 매우 체계적으로 구성되어 있다. 이를 점진적으로 배워가는 과학적인 건강법이다.\
    물질문명의 발달과 환경오염 속에서 심신이 약화된 현대인들에게 마음에는 힐링을, 몸에는 활력을 불어넣어 주고 치유력과 자생력 그리고 창의력을 강화시켜줄 수 있는 심신수련체계일 뿐만 아니라, 나아가 극치적 체력, 극치적 정신력, 극치적 도덕력을 겸비한 전인적 인간상을 만드는데 그 목적을 두고 있다. (출처: 국선도본원)\
    몸과 마음이 건강해지면, 인간이 지녀야 할 올바른 자세를 자연스럽게 터득하게 되고 사언행(思언行)이 올바르게 행해질 수 있다. 국선도 수련은 사욕(私慾)과 사심(私心)을 버리는 의지를 기르고 공심(公心)과 도심(道心)을 갖게 하는 수련법이다.\
    남녀노소를 불문하고 사상, 종교, 인종과 국경을 초월하여 누구나 실천할 수 있는 건강법으로, 자연을 따라 바르게 먹고, 마시고, 운동하고, 생각하고, 말하고 행동하여 자연의 공도(道)와 공심(公心)으로, 자연의 도리에 맞는 도심(道心)으로, 정도(正道)의 생활을 하여 건강과 기력이 넘치는 충실한 생명체를 기르는 양생법(養生道法)이라 한다. (출처: 삶의 길, 저자: 청산선사)}"
    },
    {
        'instruction': '다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 유래\n',
        'output':"국선도는 한국 민족의 기원과 역사를 같이 하는 심신 수련법으로 그 출발이 인류문명의 태동 이전으로 거슬러 올라간다. \
        옛 한국 민족은 밝고 따스하고, 온화하게 만물을 살리는 '태양'을 사모하고, 하늘의 도와 자연의 높고 큰 덕을 본받아 그 덕성을 닮고자 하늘의 밝(태양)과 가장 가까운 높은 산에 올라가\
         하늘 기운을 깊이 마시는 가운데 그 위대한 힘을 몸과 마음에 받아들이는 수련의 방법을 체득하여 밝돌법의 수련 체계를 세운 것이다. 국선도는 9,800여년 전 하늘함도인(天氣道人)으로부터\
         세상에 전파되어 상고시대를 거쳐 고조선 시대로 내려오면서 밝도, 밝받는 법, 선도, 국선도, 풍류도 등 다양한 이름으로 불려왔다. 삼국시대는 고구려의 조의선인제도, 백제의 수사제도, 신라의 화랑제도에서 국선도의 맥을 이어왔다. \
         국선도는 우주자연과 인간이 하나되는 수련법을 지향한다. 조화로운 세상을 만들어가는 국가 지도자의 도법으로서 전인적 인간을 양성하는 수련 체계를 갖추고 있기에 국가 제도적 차원, 수양단체적 차원, \
         혹은 순수 수도적 차원으로 각각 그 명맥을 이어오다 고려시대, 조선시대를 거치면서 당시 사회적 환경에 의해 산중에서 스승에서 제자로 대대로 비전으로만 전승되어 오던 국선도의 주맥을 무운도인의 대를 이은 청운도인으로부터 \
         도맥을 이어받은 청산선사가 천원기(天元紀)9715년 3월 3일(1967년 음력 3월 3일) 국선의 도법을 세상에 다시 펼치기 시작하였다."
    },
    {
        'instruction' :"다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 수련의 개요\n",
        'output': "국선도의 수련과정은 점진적이고 체계적이다. 준비운동 60여 가지의 동작에서 시작하여 단전행공으로서 다양한 동작과 함께 이루어지는 호흡수련, 그리고 40여 가지 각종 동작으로 구성된 정리운동으로 진행되며 각 개인의 수련 경과에 따라 점점 높은 단계로 승단하는 데 힘의 무리가 없는 것이 장점이다.\
수련 단계의 첫단계는 정각(正覺道) 단계로 육체적, 정신적 수련을 닦는 단계이며 하단전(下丹田)에 기가 모이고 마음이 안주할 수 있는 자리를 확립하는 중기단법(中氣丹法)과 하늘과 땅과 사람 사이의 조화를 이루는 건곤단법(乾坤丹法)과 인체의 구석 구석과 모든 경락에 기혈이 순조롭게 유통되어 대자연의 원기와 상통 될 수 있도록 하는 원기단법(元氣丹法)으로 구분된다.\
이 과정을 거치는 동안 병이 자연적으로 치유되고 건강에 대한 자신과 지식을 갖게 되는 것으로 현대사회의 백만 이상의 수련자들의 체험을 통해 알 수 있다.\
정각도 단계의 다음은 통기법(通氣法) 단계로 극치적인 육체적, 정신적 수련을 닦게 되며 임독유통을 자개시키는 단계인 진기단법(眞氣丹法), 피부로 호흡하여 천지인 합일의 경지로 들어가는 삼합단법(三合丹法), 조리단법(造理0丹法)으로 구분된다. 이 단계에서는 우주의 기운을 내 몸 안에 받아들여 완전히 기혈을 유통시키며 피부호흡을 하는 단계로 참된 진리의 문에 들어서게 되며 설명으로 이해가 불가능하다고 한다.\
마지막 단계는 선도법(仙道法) 단계로 천인묘합(妙)의 이상을 관념이 아니라 직접 몸과 마음으로 구현하는 초인적 단계이며 삼청단법(三淸丹法), 무진단법(無盡丹法), 진공단법(眞法)으로 구분된다.\
그러나 국선도에서는 수련이 최고 단계에 도달해야 어떤 효과를 보는 것은 아니라며, 자기가 수련한 만큼 즉시 효과를 스스로 체험하게 된다고 강조한다. 심신(心身)의 부조리(不調理)를 느끼어 수련하는 사람이면 하루에 한두시간씩 수련하여 반년이나 일년 안에 건강한 사람이 될 수 있으며, 첫번의 세 단계를 정각도라 하는데 그리하여 이것이 양생(養生)의 도임을 증명한다."
    },
    {
        'instruction': "다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도 수련의 내용\n",
        'output' : "일반 수련장에서의 수련은 준비운동(15분), 행공(40분), 정리 운동(25분)의 순서로 진행된다. (총 1시간 20분))\n\
1. 준비운동 (기혈순환 유통법: 60여 동작) 돌단 숨쉬기의 몸동작(단전행공)에 앞서 기의 순환을 도와주는 전 동작으로 사람의 말단부에서 기중심부 그리고 온몸의 순서로 관절과 근육을 여러 각도에서 무리함 없이 고루 움직여 주어 조화와 균형을 이루게하는 운동이다.\
2. 단전행공 (433가지의 동작) 동작을 하면서 숨쉬기를 하여 공력을 쌓는다는 의미로 행공이라 한다. 수련법에 따라 각 단계에 맞는 동작과 숨쉬기를 한다. 숨쉬기와 더불어 온몸을 고르게 움직여 그 기운과 체력을 더욱 강화시켜 강인한 육체와 정신에 이르게하는, 국선도 특유의 도법 원리를 실천하는데 있어서 행공이 매우 중요함이 강조된다.\
3. 정리운동 (정체 운동) 행공이 끝나고 나서 행하는 동작으로 전신의 기혈을 원활히 하여주고 근골과 기육, 정동맥, 신경강화를 도와주는 것이므로 반드시 해야 한다. 모든 동작은 처음에는 모양과 방법을 익히는데 중점 노력을 기울이던 것이 반복에 반복을 거듭함에 따라 점차 익숙해지고 여유가 생겨 호흡과 운기가 가능해지면 단순한 스트레칭의 효과를 넘어서 국선도 동작과 숨쉬기의 차별화된 차원 높은 맛과 멋을 느낄 수 있게 된다."
    },
    {
        'instruction' : '다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도 밝돌법을 꾸준히 수련했을 경우 얻을 수 있는 효과\n',
        'output' : "국선도 밝돌법을 1,2년 정도만 꾸준히 수련해도 일반적으로 얻을 수 있는 효과들을 조금 더 구체적으로 나열하면 다음과 같다.\n\
1. 신체적인 효과 체력이 증강되며 항상 최고의 컨디션을 유지할 수 있다. 만성피로에서 해방되며 등산시에도 숨이차지 않다. 피하지방의 연소가 활발해져 비만증이 근본적으로 사라진다. 시장기를 느끼고 소화가 잘 된다. 잠이 잘 오고 숙면할 수 있다. 대소변이 순조로워 진다. 피부가 고와지고 얼굴이 맑고 기품이 생긴다. 음주와 흡연의 자제가 쉬워진다. 각종 성인병 증상을 개선, 치료할 수 있다. 손발시림, 저림, 요통 두통 등 몸의 이상이 빠르게 없어진다.\
2. 정신적인 효과 마음이 상쾌하고 평화로우며 너그러워 진다. 인내심과 의지력이 강해진다. 삶에 대한 무료함, 우울증 등이 해소되며 삶에 대한 의욕이 생긴다. 불안감, 소심증, 열등감이 사라지고 모든 일에 자신감이 생긴다. 두뇌가 맑아져 집중력이 크게 향상된다. 누적된 스트레스로 부터 자연스럽게 해방된다. 국선도 호흡을 하면 우리가 미처 상상하지 못할 정도의 건강상의 효과가 일어난다는 것은 이미 수많은 사례들이 실증하고 있다."
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 활용\n',
        'output':"\
1. 무병장수: 긴 세월에 걸쳐 갈고 닦져진 무병장수와 불로장생의 비법으로 동양학 예지의 결정체이므로 수련을 통하여 무병장수의 안락 생활을 향유할 수 있다.\
2. 인격도야: 강건한 체력과 고귀한 인품을 겸비하게 하는 심신수련법으로 사회적 중책을 성공적으로 완수있도록 도와준다.\
3. 자연치유: 몸과 마음의 모든 병폐를 뿌리째 뽑아 낼 수 있는 근본적 수련법이다.\
4. 깨달음: 우주와 인생에 대한 참된 진리를 깨닫게 되어 삶의 의미와 희열을 느끼며 살게 된다.\
5. 자기개발: 체력과 정신력과 도덕력을 고도로 발달시켜 각 분야의 유능한 인재로 성장할 수 있다.\
6. 디톡스; 전문직종의 과도한 디지털 노출과 두뇌활동에서 오는 신체적 부담과 정신적 고통을 효과적으로 해소할 수 있는 수련법이다.\
7. 무술과 내공: 모든 체육과 무술의 과도한 활동에서 오는 부상과 후유증을 방지하고 외적인 기량을 증진시켜 주는 정통 내공법이다.\
8. 예술과 수양: 음악 무용 그림 서예 등 다양한 문화예술활동의 중요한 기초가 되는 호흡과 맑은 집중력 그리고 충만한 에너지를 제공한다.\
9. 스트레스관리: 밝(태양)의 광명정대한 정신과 막강한 힘을 호흡과 명상을 통해 매일 받아 들이는 습관은 수많은 스트레스의 상황속에서도 마음의 태평성대를 누릴 수 있는 정신적 역량을 이룰 수 있게 한다."
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도 훈\n',
        'output':'훈(訓)\n\
정심(正心) : 하늘사람의 참되고 올바른 마음으로\n\
정시(正視) : 하늘의 모든 것을 올바로 살펴보고\n\
정각(正覺) : 참된 진리를 올바로 깨닫고 얻어가져\n\
정도(正道) : 하늘사람의 도리를 다하여 올바르게 살 수 있는 길로\n\
정행(正行) : 추호의 잘못된 길로 가지 말고 올바른 길을 가라.\n\
수련하기 전에 항상 외치는 것으로 국선도 수련자가 가져야 할 마음가짐에 대한 지침이라 볼 수 있다.'
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도 도인도선\n',
        'output':'도인도송:우주만유(宇宙萬有)가 현실(現實)을 유지(維持)하는 것은 중기(中氣)의 운용(運用)이니 중기(中氣)는 음양(陰陽)이 합실(合實)한 중심원리(中心原理)이니 유도(儒道)의 윤집궐중(允執厥中)과 선도(仙道)의 포일수중(包一守中)과 불도(佛道)의 중도(中道)가 모두 중(中)을 집수(執守)한다. 음양(陰陽)이 변화하며 수화(水火)가 승강(昇降)하며 기혈(氣血)이 순환(循環)하는 것이 모두 중앙오십토(中央五十土)의 중기단합력(中氣團合力)에 의한 것이니 인체의 중(中)은 비장(脾臟)과 위(胃)가 토(土)니 중기(中氣)는 실(實)로 중대(重大)한 생리(生理)이며 일신(一身)의 음양(陰陽)은 단합(團合)하면 생존(生存)하고, 분리(分離)하면 사망(死亡)하는 것이니 중기(中氣)는 유음유양(有陰有陽)의 진실성(眞實性)이요, 이음조양(理陰調揚)의 조절성(調節性)이요, 유무변화(有無變化)의 대사성(代謝性)이니 하단전(下丹田)에서 생동(生動)의 힘이 간(肝)으로 나와 생신(生新)한 기혈(氣血)이 되어 내 몸을 보양(保養)하는 것이므로 중기단법의 50토(土) 동작의 단법(丹法)을 수도(修道) 초공(初功)을 삼고 천(天)의 신공(神功)과 지(地)의 덕기(德機)와 오기(五氣)의 기운(氣運)이 합성(合成)하여 만물이 화생(化生)하는 것이니 건곤기(乾坤氣)는 만물화생(萬物化生)의 조종적(祖宗的) 근원(根源)이 되는 것이며 오인(吾人) 생명체는 정.기.신(精,氣,神)인즉 이를 삼원(三元)이라 하고 건(乾)은 천원(天元), 곤(坤)은 지원(地元)이며 오인(吾人)은 인원(人元)이니 건곤지중(乾坤之中)에 소장(所藏)되어 있는 것이 인원(人元)인 고(故)로 수도(修道)에는 건곤단법(乾坤丹法)을 제2(第二)의 행공(行功)을 삼고 천(天)의 오운(五運)과 지(地)의 육기(六氣)의 작용으로 일어나는 육기(六氣)는 수화합실력(水火合實力)인즉, 수화합실력(水火合實力)은 인신(人身)의 원기(元氣)요 체온(体溫)이 되는 고(故)로 천도(天道)의 삼백육십오도(三百六十五度)와 오인(吾人)의 삼백육십오골절(三百六十五骨節)이 유(有)하므로 원기(元氣)단법을 우주적(宇宙的) 입장(立場)에서 오인(吾人)이 행공(行功)하여야 하는 고행(苦行)인 것이니 지금으로부터 고요한 적경(寂境)에서 일신일심법의 정법, 건의 갑법'
    },
    {
        'instruction': '다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도 선도주\n',
        'output' : '선도주\n\
정각도원(正覺道原) : 진리의 근원을 내가 바르게 깨달아\n\
체지체능(體智體能) : 내가 지혜와 능력을 얻어가져\n\
선도일화( 道一和) : 하늘사람 진리에 하나가 되어\n\
구활창생(救活蒼生) : 하늘 안의 모든 생명체를 구하리.'
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 단체\n',
        'output':"4.1. 국선도의 본원(本源)\n\
국선의 맥을 이어 온 선인들의 뜻을 받들어 1967년 청산선사께서 하산하여 국선도의 사회 보급이 시작되었다. 1970년 음력 3월 15일 청산선사께서 국선도 본원을 개원하여 본원 수련장을 기반으로 사회 각처에 보급 활동을 시작하였는데 국선도법의 근원이 되는 국선정신을 보존하고 도법의 올바른 가르침을 펼치는 국선도의 원천이라는 의미에서 본원(本源)이라 하였다.\
국선도의 현대사를 보면 다양한 단체로 분화발전한 것을 볼 수 있다.\
그럼에도 불구하고 명확한 법통과 도맥을 유지해오고 있으며, 국선도의 현대사에 대해서는 국선도본원의 밝문화연구소에서 출판한 '청산, 갈대밭에 콩심다' (청산뮤지엄 전시내용을 담은 기록)에 자세하게 나와있다.\n\
4.2. 국선도의 고유권한\n\
청산선사가 책자, 육성, 영상물, 사진, 친필 메모 등으로 밝힌 국선도(밝돌법) 수련에 관련한 모든 지적재산 및 국선도(밝돌법)의 명칭, 마크와 심볼 등의 상표 일체는 국선도본원의 (사)국선도법연구회가 고유 권한을 보유하고 유일하게 사용 권한을 가지고 있으며, 국선도법의 정통성과 원뜻을 잃지 않도록 보존하고 관리되고 있다.\
국선도본원의 명의로 발급되는 수료증, 승단증, 도단증, 지도자 임명장 등의 자격증서는 국선도본원 세계본부가 세계적으로 발급하는 유일한 공인 자격증입니다. 그 고유성을 보존하기 위하여 국선도본원에서는 2022년부터 도단증과 지도자 자격증을 블록체인 증서로 발급하기 시작하였다."
    },
    {
        'instruction' :'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 기혈순환유통법\n',
        'output':'기혈순환유통법(준비운동)\n\
훈을 먼저 외치고 단전행공에 들어가기 전에 몸을 풀어주는 동작들이다. 말이 준비운동이지 처음 하는 사람은 웬만한 운동 저리 가라 할 만큼 힘들다. 초인적인 고난도 동작이 있어서 그런 것은 아니고 평소에 취하지 않던 동작을 취하고 풀리지 않았던 곳곳의 근육, 관절, 인대 등을 풀어주기 때문에 익숙해지기 전까지는 힘들 수 있다.\
기혈순환유통법만 해도 건강만큼은 지킨다고 할 정도로 기혈 순환과 양생에 크게 도움이 되는 운동이다. \
기혈순환유통법 시 주의점은 다음과 같다. \n\
항상 허리는 반듯하게 펼 것. \n\
모든 동작을 할 때 숨을 마시고 멈춘 뒤(지식) 기운이 해당 부위로 통한다는 생각을 하며 동작을 행하고 동작이 끝나면 숨을 뱉을 것 \n\
다만 숨이 모자라면 억지로 멈추지 말고 알아서 호흡을 조절해가면서 할 것 \n\
동작이 안 된다고 하여 과한 힘을 주지 말고 은은한 힘을 준다고 생각할 것 \n\
절대 무리하지 말 것. 하다가 다칠 삘이면 그냥 안 하고 넘어가는 것이 억지로 하는 것보다 훨씬 좋다. 안 하고 넘어갈 때에는 생각으로만 하면 된다.'
    },

    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도 단전호흡의 개요\n',
        'output': '국선도 단전호흡의 개요\n국선도에서는 단전호흡을 돌단자리 숨쉬기라 부른다. 청산선사에 따르면 돌단자리는 하단전의 순 우리말로써 단단한 기운이 돌아가는 자리라 하여 그리 부른다고 한다. 국선도의 단전호흡은 한마디로 정기신 삼단전 이단호흡(精氣神 三丹田 二段呼吸)이라 하며, 정기신 삼단전은 기가 머무르는 상단전, 신이 머무르는 중단전을 모두 정이 머무르는 하단전에 마음으로 모은다는 뜻이다. 이단호흡은 일반적으로 호흡만 하는 것이 아니라 개요에서 설명한 것처럼 호와 흡 사이에 지식(止息)이 들어가기 때문에 호흡과 지식, 2단으로 이루어진 호흡을 한다는 뜻이다.'
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도 단전호흡에서 지식에 대한 오해\n',
        'output':'지식(止息)에 대한 오해\n지식이란 한자 그대로 풀이하면 숨을 그치는 것인데 이것은 일반적으로 우리가 잠수할 때와 같이 숨을 멈추는 것과는 다르다. 호흡을 고요하게 하다보면 평소에 숨쉴 때는 거의 못 느끼지만 호에서 흡으로, 흡에서 호로의 전환이 점점 분명해지게 된다. 이 내쉬는 것도 들이쉬는 것도 아닌 그 잠깐의 상태가 바로 지식의 기초라고 볼 수 있으며 국선도에서 말하는 숨을 멈춘다는 것은 바로 이 상태를 말한다. 마음이 고요한 경지에 들어가고 숨이 점점 깊어질수록 호에서 흡, 또 흡에서 호로 전환되는 시간 또한 점점 길어지는데 건곤단법에 이르러서는 흡 - 지 - 호 - 지 의 길이가 모두 5초씩으로 같아지는 것이다. 또한 원기단법 이상에서는 흡의 길이와 지의 길이가 끝도 없이 늘어나는 것이다. 청산선사는 흡하고 멈출 때 이미 숨은 나가고 있는 것이며 호하고 멈출 때 숨은 이미 들어오고 있는 것이라 한 바 있다.'
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도 호흡\n',
        'output':'구체적 호흡법\n\
도장마다, 단체마다 처음에 가면 가르치는 방법이 조금씩 다르나 그 요체는 같다고 볼 수 있다. 다만 여기서는 객관성을 위해 청산선사가 직접 국선도 2권에 저술한 호흡법에 대해 기술하고 설명하겠다.\n\
국선도에 입문하면 천천히 준비운동을 하고, 준비 운동이 끝나면 고요히 마음을 가라 앉히는 조심(調心)을 하고 누워서 돌단자리(하단전)로 숨쉬는 것을 연습하게 되는 것이다.\n\
그리고 돌단자리 숨쉬기가 임의로우면 숨을 고르는 조식(調息)을 하게 된다.\n\
숨을 고르는 조식(調息)이 임의로우면 기를 축기시키는 방법으로 천기(공기)와 지기(섭취하는 기)가 돌단자리에 모이도록 기를 모이게 한다는 생각으로 배꼽 위의 상체는 고요한 가운데 거의 없다는 생각을 하고 관원, 중극, 곡골, 전음, 회음, 장강, 신장의 순서대로 돌린다는 염을 하되 그림과 같이 하고, 항문을 움추리는 듯하고 숨을 마실 때에는 은은히 힘이 발에서부터 올라오게 하는 것이다.'
    },

    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도 호흡의 순서\n',
        'output':'이렇게 보면 되게 어렵기는 한데 순서를 정리하여 쉬운 말로 풀어보면 다음과 같다.\n\
마음을 가라앉힌다.\n\
마음이 가라앉으면 하단전으로 숨을 쉰다.\n\
하단전으로 숨쉬는 것이 되면 숨을 고르게 한다.\n\
숨을 고르게 하는 것이 임의로우면 코에서 들어오는 공기와 소장에서 흡수된 영양분이 하단전에 모인다는 생각을 하며 배꼽 위는 없다는 생각을 하고 그 모인 기운이 아랫배 3cm 아래 - 치골 - 회음(성기와 항문 사이) - 꼬리뼈 - 신장과 같은 방향, 즉 몸을 왼쪽에서 보면 반시계 방향으로 기운을 돌린다는 생각을 하되 항문을 살짝 움츠리고 발에 은은한 힘을 준다.'
    },

    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도 호흡의 순서\n',
        'output':'다시 저 순서를 풀어서 설명하면 다음과 같다.\n\
마음을 가라앉힌다는 말이 추상적일 수 있는데 도장에 가보면 대개 눈을 감고 단전에 마음을 두고 그냥 가만히 바라보라고 한다. 처음에는 좀이 쑤시고 좀처럼 집중할 수가 없지만 그걸 참고 조금만 있으면 마음이 고요해지는 단계가 온다. 이것이 조심(調心)이다.\n\
하단전으로 숨을 쉰다는 것 또한 어떻게 하라고? 할 수 있는데 사실 마음이 잘 가라앉으면 저절로 숨이 아랫배까지 내려간다. 물론 삶에 찌든 현대인들은 그런 거 없기 때문에 일선 도장에서는 몇 가지 방편을 쓰는데 대표적으로 쓰는 방법이 꼬리뼈로 숨을 마셔 아랫배를 밀고 숨을 내쉴 때도 숨이 꼬리뼈로 나간다는 생각을 하는 것이다. 꽤 잘 통하는 방법이라고 하니 해보면 좋다.\n\
숨을 고른다는 말은 내쉬는 숨과 마시는 숨의 길이를 같게 하라는 말이다. 이 모든 과정이 중기단법을 하기 위한 과정이니 5초-5초, 선도주 기준으로는 4박자-4박자로 호흡을 하면 된다.\n\
마지막 방법은 기운을 쌓는 방법인데 숨을 마실 때 기운을 돌리되 나선형으로 점점 가운데로 모이게 하고 내쉴 때 가운데 점으로 기운을 꽉 응축시킨다는 생각을 하면 된다.\n\
보통 도장에서는 초급자는 단전행공에 들어가지 않고 바로 누운 자세, 앉은 자세, 선 자세를 번갈아 가며 3번까지 되도록 연습을 시킨다. 그리고 단전행공에 들어간 사람은 누워서 1번부터 4번까지 되도록 하되 안 되면 행공 중에라도 숨을 가다듬어 자기 단법에 맞는 호흡이 될 때까지 가다듬게 된다. 그야말로 고행이다. 원래 산중수련 같았으면 20분이고 30분이고 가만히 마음을 모아 호흡이 바르게 된 뒤에 행공에 들어가는 것이지만 일선 도장에서는 1시간 20분에 맞춰 해야 하기 때문에 그런 거 없이 바로 들어가는 것이다.\n\
국선도 2권에서 청산선사는 이 모든 과정이 된 후에 중기단법을 들어가라고 했으나 일선 도장에서는 사실 2번까지 됐으면 바로 중기단법을 시킨다. 왜냐하면 같은 것을 너무 오래 시키면 사람들이 지루해해서 도장에 안 나오기 때문이다.'
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도 호흡시 주의사항\n',
        'output':'호흡 시 주의사항\n\
호흡을 힘을 주거나 무리해서 하면 안 된다. 자연스럽게 단전호흡이 되어야 한다. 참을성을 가지고 위의 순서를 잘 지키면 그럴 일은 없다.\n\
마음이 가라앉지 않았는데 아랫배를 내밀려고 하면 본능적으로 힘을 주어 윗배를 눌러 아랫배가 상대적으로 더 나오게 하게 되어 있다. 이렇게 되면 단전호흡의 온갖 부작용들이 나타나는 것이다.\n\
항상 마음을 먼저 가라앉히고, 아랫배로 숨이 잘 쉬어지는지 보고, 천천히 숨의 길이를 조절해 보는 것이다.\n\
항상 내쉬거나 들이쉴 때 최대치의 80%까지만 한다고 생각하는 것이 좋다. 왜냐하면 한계까지 들이쉬거나 내쉬게 되면 저절로 배가 긴장하게 되어 결국 앞에서 마음을 가라앉힌 것이 허사가 되기 때문이다.\n\
위의 지식 설명에서도 설명했지만 항상 호에서 흡, 흡에서 호로 전환될 때는 갑작스럽게 전환하는 것이 아니라 부드럽게 전환되어야 한다.\n\
배꼽 위의 상체는 항상 이완되어 있어야 한다. 배꼽 위의 어디라도 힘이 들어가거나 긴장되면 숨이 아랫배까지 내려가지 않는다. 이 상태에서 억지로 아랫배를 내밀려고 하면 온갖 부작용들이 따라 붙는 것이다.'
    },
    {
        'instruction' : '다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 단전행공의 단계와 단법\n',
        'output':'단전행공\n\
정각도(正覺道) 단계: 중기단법(中氣丹法), 건곤단법(乾坤丹法), 원기단법(元氣丹法)\n\
통기법(通氣法) 단계: 진기단법(眞氣丹法), 삼합단법(三合丹法), 조리단법(造理丹法)\n\
선도법( 道法) 단계: 삼청단법(三淸丹法), 무진단법(無盡丹法), 진공단법(眞空丹法)\n\
국선도 수련의 핵심이자 꽃이다. 다른 모든 수련법은 이 단전행공이 잘 이루어지도록 하는 보조 수련법이라 할 수 있고 수행하고 있는 단전행공의 단계에 따라 띠를 부여 받게 된다.\n\
도장에 처음 가게 되면 바로 중기단법부터 시작하는 것이 아니라 누운 자세, 앉은 자세, 선 자세에서 모두 단전호흡을 원활히 할 수 있도록 호흡을 고르는 법을 먼저 배우게 된다. 세 자세에서 단전호흡이 임의롭게 되면 그때부터 본격적인 단전행공 수련에 들어간다고 보면 된다.\n\
모든 단법 공통적으로 단전행공을 할 때 주의사항이 있다. 한 동작에서 다른 동작으로 전환할 때에는 단전호흡이 끊기지 않도록 고요하고 천천히 바꾸어야 한다. 또한, 동작을 정확하게 하는 것도 중요하나 호흡에 무리가 없도록 하는 것이 중요하다. 정확한 동작을 하기 위해 과도한 힘을 주고 호흡이 깨지면 좋지 않다.'
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 정각도 단계\n',
        'output':'타 수련법과 국선도가 차별점을 두는 단계라고 볼 수 있다. 보통 수련 단체들 같은 경우 바로 정신수련법으로 돌입하나 국선도에서는 이 정각도 단계를 통해 육체를 완전히 가다듬고 닦은 뒤에 본격적인 정신수련법인 통기법으로 넘어가게 되어 있다. 그래서 호흡 수련에서 올 수 있는 각종 부작용이 매우 덜한 편이다.\n\
육체를 단련하는 단법답게 중기단법 50동작, 건곤단법 23동작, 원기단법 360 동작으로 동작 수가 매우 많고 윗 단계로 갈수록 아크로바틱한 동작들을 해내면서 아주 깊은 단전호흡을 해야 한다. 정각도 단계만 놓고 보면 요가와 매우 비슷하다.'
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 중기단법 단계\n',
        'output':'원래 중기단법은 50동작이나 시중 수련장에서 40분 내에 마치는 것은 불가능하므로 전편 25동작, 후편 25동작을 보통 수련하게 되며 집에서 수련하거나 도장 혹은 연수원 등에서 특수 프로그램이 있으면 50동작을 한번에 수련하기도 한다.\n\
중기단법에서는 호의 길이와 흡의 길이를 각 5초로 선도주에 맞추어 동일하게 하며 본격적으로 단전호흡을 배워나가는 단계라고 보면 된다. 이때는 단전자리가 잡히고 아랫배로만 호흡하는 것이 원활히 되고, 호흡이 점점 깊어지며 호와 흡 사이에 지식이 점점 길어지는 것을 느끼게 된다.'
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 건곤단법\n',
        'output':'중기단법의 모든 동작이 임의롭게 되고 지식이 점점 길어져 호흡의 길이와 지식의 길이가 같아지는 단계에 도달하면 건곤단법을 수행하게 된다. 그러나 도장에서는 그냥 일수만(보통 3개월) 채우면 건곤단법으로 승단시켜주기도 하는데 이럴 경우 건곤단법에서 뺑이를 많이 쳐야 한다.\n\
건곤단법에서는 흡-지-호-지의 길이가 모두 각각 5초가 되어 결국 20초에 한번 호흡을 하는 경지이다. 지식이 이 정도로 길어지면 조금씩 기가 쌓이기 시작하는데 곤의 신법에서 하루에 1번 임독유통을 마음으로만(축복받은 체질은 이때 실제 임독유통이 이루어지기도 한다고 한다.) 시도해 보는 것이다.',
    },
        {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 원기단법\n',
        'output':'원기단법은 360동작으로 이루어져 있다. 청산선사에 따르면 원래는 365 동작이었으나 마지막 5동작은 진기단법에서 주력하여야 하기 때문에 360동작으로 개편했다. 다른 단법과 달리 이 모든 동작을 한번에 다 하는 것은 아니고 한 달의 30일을 의미하는 30절로 나누어 각 12절기를 상징하는 12동작을 차례로 수련하게 된다.\n\
원기단법에서는 흡과 지를 할 수 있는 한 최대한 길게 하고 호와 지 혹은 호는 몸에서 요구하는 대로 한다. 원기단법에서는 임독유통을 자주 시도하며 12경, 14경, 365혈 유통까지 시도하게 된다. 원기단법에서는 흡지가 매우 길어지기 때문에 축기 또한 많이 되어 실제로 기운이 일어나 그것을 움직일 수 있어야 하고 실제적인 임독유통을 할 수 있어야 한다.',
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 통기법\n',
        'output':'진정한 도의 입문 단계이다. 본격적인 정신수련 및 초자연적 수련으로 들어가게 된다.',
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 진기단법\n',
        'output':'진기단법에서는 원기단법에서 하던 흡지의 길이를 40초, 50초까지 늘리게 된다. 또한, 임독유통을 이전에는 몇 가지 동작에서 시도를 하는 정도였다면 진기단법에서는 임독유통이 자유자재로 될 때까지 수련하며 영체를 몸에서 분리해내어 같이 수련을 하기 시작한다.',
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 삼합단법\n',
        'output':'여기서부터는 사실 매우 초자연적인 현상이 일어나지 않으면 수련이 불가능한 단계라 이론적인 설명만 적도록 하겠다. 진기단법에서 흡지가 매우 길어지게 되면 숨을 뱉지 않았는데도 내쉴 때 뱉을 숨이 얼마 남지 않게 된다. 이것이 피부호흡의 시작인데 삼합단법에서는 수련 중에 코로 하는 호흡을 최대한 줄이고 점점 피부호흡으로 대체해 나가는 단법이다.',
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 조리단법\n',
        'output':'조리단법에서는 피부호흡만으로 기운을 모아 임독, 12경, 14경, 365혈 기운을 모두 유통시키는 단법이다.',
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 선도법\n',
        'output':'하늘과 하나가 되는 단법',
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 삼청단법\n',
        'output':'한없이 많은 수로 분신을 하여 하는 수련',
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 무진단법\n',
        'output':'몸과 마음을 따로 분리하여 한없이 많은 수로 나누어 하는 수련',
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 장부강화운동\n',
        'output':'보통 도장에서는 정리운동 후에 하게 되는데 여러 가지 동작을 취하면서 단전의 기운을 해당 장부로 보내어 장부를 강화시키는 운동이다.',
    },
    {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요..\n주제:국선도의 진공단법\n',
        'output':'몸과 마음을 끝없이 흩어 먼지도 남지 않게 나누었다가 다시 합치는 수련\n준비운동이 행공을 하기 전에 몸이 부드럽게 이완되도록 해주는 운동이었다면 정리운동은 행공을 하면서 모아두었던 기를 전신에 뻗치고 돌려주는 운동이다. 준비운동보다 난도 높은 동작들이 많으나 행공이 잘 이루어졌다면 오히려 힘이 나 더 쉽게 되는 현상을 경험할 수 있다. 정리운동의 주의사항 또한 기혈순환유통법과 같다. 항상 동작 시에 숨을 멈추고 기를 돌려야 하며 절대 무리해서는 안 된다.',
    },
        {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 내기전신행법\n',
        'output':'보통 줄여서 기신법이라고 많이 부른다. 신장, 방광, 심장, 소장, 간장, 쓸개, 폐, 대장, 비장, 위장을 각각 강화하게 되어 있다. 꽤 힘든 동작들을 지식 상태에서 해야 하기 때문에 숨이 찰 수 있다. 꼭 동작 사이 사이에 심호흡을 하여 숨을 완전히 고르고 다음 동작으로 넘어가도록 하자.',
    },
        {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 천화법\n',
        'output':'내기전신행법의 업그레이드판이라고 볼 수 있다. 보통 천화법은 원기단법이 끝날 때즈음부터 시작한다고 한다.',
    },
        {
        'instruction':'다음 주제에 대하여, 위키피디아에 있는 정보를 어떠한 수정 없이 사실대로 말해주세요.\n주제:국선도의 기화법\n',
        'output':'국선도의 무술이다. 관련 자료가 많지 않고 일반 수련장에서도 적극적으로 지도하지 않는다. 종류도 많고 무기술까지 있다고 하니 자세한 사항은 각 도장 사범에게 문의해보자.',
    },
]

In [6]:
len(data)

35

In [7]:
import json
with open("/content/drive/MyDrive/mygpt-lecture/data_kochatgpt/domain_adaptive_kuksundo_pretrain.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in data: file.write(json.dumps(i, ensure_ascii=False) + "\n")

# API test

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 1.2 MB/s eta 0:00:00


In [ ]:
import openai

In [ ]:
openai.api_version=""
openai.api_key="### INPUT YOUR KEY ###"
response = openai.ChatCompletion.create(
    model='gpt-3.5-turbo',
      messages=[
        {"role": "user", "content": "안녕? 넌 이름이 뭐니?"}],
    max_tokens=512,
    temperature=0,
)

print(response['choices'][0]['message']['content'])

안녕! 나는 OpenAI의 GPT-3 모델이야. 이름은 없어, 그냥 GPT-3라고 불러줘! 어떻게 도와줄까?


### ChatGPT를 활용하여 도메인 특화 질의 응답 생성(Stage 1 데이터)



In [ ]:
from tqdm import tqdm

In [ ]:
qa_data = []
error_data = []

In [ ]:
PROMPT="다음 주제와 내용을 보고 답변 할 수 있는 질문과 정답 쌍을 10개 만들어줘.\n주제:{title}\n내용:{content}\n\
답변은 반드시 아래 형태를 띄어야 해.\n\
Q1:질문\n\
A1:정답\n\
\n\
Q2:질문\n\
A2:정답\n\
\n\
Q3:질문\n\
A3:정답\n\
\n\
Q4:질문\n\
A4:정답\n\
\n\
Q5:질문\n\
A5:정답\n\
...\n\
Q10:질문\n\
A10:정답\n\
"

for a in tqdm(data):

    title = a['instruction'].split("주제:")[-1]
    content = a['output']
    try:
        response = openai.ChatCompletion.create(
        engine='gpt-3.5-turbo',
        messages=[
            {'role':"system","content":"You are a helpful assistant."},
            {'role':"user","content":PROMPT.format_map({'title':title, 'content':content})},
            ]
        )
        qa_data.append(response['choices'][0]['message']['content'])
    except Exception as e:
        error_data.append(a)
        print(title)
        print(e)

 31%|███▏      | 11/35 [14:29<1:20:12, 200.52s/it]

국선도의 단체

Request timed out: HTTPSConnectionPool(host='eastus-instance-nlu.openai.azure.com', port=443): Read timed out. (read timeout=600)


 34%|███▍      | 12/35 [24:30<2:03:30, 322.20s/it]

국선도의 기혈순환유통법

Request timed out: HTTPSConnectionPool(host='eastus-instance-nlu.openai.azure.com', port=443): Read timed out. (read timeout=600)


100%|██████████| 35/35 [45:58<00:00, 78.83s/it]


In [ ]:
instruction_data=[]
for a in qa_data:
    for i, b in enumerate(a.split("\n\n")):
        i +=1
        q = b.split("Q"+str(i)+":")[1].split("\nA"+str(i)+":")[0].lstrip()
        ans = b.split("A"+str(i)+":")[1].split("\n\nQ"+str(i)+":")[0].lstrip()
        instruction_data.append({'instruction':q,'output':ans})

In [ ]:
len(instruction_data)

350

In [ ]:
import random
test_data=[]
train_data=[]

for a in instruction_data:
    if random.random() < 0.1:
        test_data.append(a)
    else:
        train_data.append(a)

In [ ]:
import json
with open("/content/drive/MyDrive/KoChatGPT/data_kochatgpt/kuksundo_instruction_train.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in train_data: file.write(json.dumps(i , ensure_ascii=False) + "\n")

with open("/content/drive/MyDrive/KoChatGPT/data_kochatgpt/kuksundo_instruction_test.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in test_data: file.write(json.dumps(i, ensure_ascii=False) + "\n")

# Stage 2의 RM 데이터 생성

### Data Format

data = [{'prompt':"질문", 'chosen':"더 우수한 답변", 'reject':"덜 우수한 답변"}]

In [ ]:
len(data)

35

In [ ]:
print(data[0]['output'])

국선도(밝돌법)는 9,800년의 역사와 전통을 가진 것으로 알려진 한국 고유의 심신 수련법으로, 남녀노소, 인종, 종교, 질병 유무와 무관하게 누구든지 심신 수련을 통해 건강한 몸, 마음과 정신을 가진 전인적인 인간으로 거듭날 수 있도록 돕는 자기계발 수련법이다.    9단법 37단계 430여 개 동작으로 구성된 점진적이고 체계적인 단전행공(丹田行功)의 도법(道法)으로 한국 고유의 명상과학 프로그램이다.    심신수련과 인재 양성의 도법으로 약 9,800년 전부터 전해져 내려와 고조선의 소도, 고구려의 조의선인제도, 신라의 화랑도 등으로 불렸고, 고려 이후에는 산중 도법으로 전해 내려오다가 산중 수도인 청산(靑山, 본명 고한영)이 스승 청운도인의 "모두의 유익을 위해 도법을 전하라"는 명을 받고 1967년에 사회로 나와 국선도(밝돌법)를 현대 인류 사회에 다시 소개하였다.    밝돌법은 국선도의 본래 이름으로, ‘밝’은 생명의 에너지(태양, 밝음), ‘돌’은 순환하는(돌고 도는) 우주 자연의 법칙을 뜻하며, ‘밝돌법’은 우주 질서의 순리에 맞게 생명을 양생(養生)하는 수련방법을 의미한다.    국선도는 상고시대부터 전해오는 한국의 고유한 전통 심신수련법이다.


### 좋은 답변 만들기

위와 동일하나, temperature만 변경

In [ ]:

import time
from tqdm import tqdm

qa_data = []
error_data = []

In [ ]:
PROMPT="다음 주제와 내용을 보고 답변 할 수 있는 질문과 정답 쌍을 10개 만들어줘.\n주제:{title}\n내용:{content}\n\
답변은 반드시 아래 형태를 띄어야 해.\n\
Q1:질문\n\
A1:정답\n\
\n\
Q2:질문\n\
A2:정답\n\
\n\
Q3:질문\n\
A3:정답\n\
\n\
Q4:질문\n\
A4:정답\n\
\n\
Q5:질문\n\
A5:정답\n\
...\n\
Q10:질문\n\
A10:정답\n\
"

for a in tqdm(data):

    title = a['instruction'].split("주제:")[-1]
    content = a['output']
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                {'role':"system","content":"You are a helpful assistant."},
                {'role':"user","content":PROMPT.format_map({'title':title, 'content':content})},
                ],
            temperature=1,
        )
        qa_data.append(response['choices'][0]['message']['content'])
        time.sleep(1)
    except Exception as e:
        error_data.append(a)
        print(title)
        print(e)

100%|██████████| 35/35 [12:07<00:00, 20.78s/it]


In [ ]:
print(qa_data[1])

Q1: 국선도는 언제부터 전해오는 전통 심신수련법인가요?
A1: 상고시대부터 전해오는 한국의 전통 심신수련법입니다.

Q2: 국선도는 어떤 수련법으로 알려져 있나요?
A2: 국선도는 삼단 단전 이단 호흡을 통해 대기와 합일시키는 묘법으로 알려져 있습니다.

Q3: 국선도는 어떤 목적을 가지고 있나요?
A3: 국선도는 심신을 강화하여 치유력, 자생력, 창의력을 강화하고 극치적 체력, 정신력, 도덕력을 겸비한 전인적 인간상을 만드는 것을 목적으로 합니다.

Q4: 국선도는 어떤 혜택을 주는 수련법인가요?
A4: 국선도는 마음에는 힐링을, 몸에는 활력을 불어넣어 주고 치유력과 자생력을 강화할 뿐만 아니라 올바른 자세를 터득하게 하고 사상과 종교, 인종과 국경을 초월하여 누구나 실천할 수 있는 건강법입니다.

Q5: 국선도가 어떤 효과를 가지고 있는지 설명해주세요.
A5: 국선도는 몸과 마음의 건강을 도와 올바른 자세를 가지게 하고 의지를 기르며 공심과 도심을 갖게 합니다.

Q6: 국선도는 누구에게 실천할 수 있는 건강법인가요?
A6: 국선도는 남녀노소를 불문하고 누구에게나 실천할 수 있는 건강법입니다.

Q7: 국선도에는 몇 개의 몸 동작과 호흡법이 체계적으로 구성되어 있나요?
A7: 국선도에는 수백 개의 몸 동작과 정교한 호흡법이 체계적으로 구성되어 있습니다.

Q8: 국선도는 어떤 생활을 하여 건강과 기력을 강화하는 양생법으로도 불리나요?
A8: 국선도는 자연의 공도와 공심, 자연의 도리에 맞는 도심, 정도의 생활을 하여 건강과 기력이 넘치는 충실한 생명체를 기르는 양생법으로도 불립니다.

Q9: 국선도의 원래 명칭은 무엇인가요?
A9: 국선도의 원래 명칭은 밝돌법으로 알려져 있습니다.

Q10: 국선도는 어떤 역사를 가지고 있나요?
A10: 국선도는 상고시대부터 전해져 온 배달민족의 혼과 민족정기가 담긴 한국 고유의 수련법이며, 삼국시대 이전에는 널리 행해졌으며, 삼국시대 이후로는 깊은 산중에서만 전해져 왔으나 1967년에 청운도인 청산선사로부터 다시 전파되

In [ ]:
instruction_data=[]
for a in qa_data:
    for i, b in enumerate(a.split("\n\n")):
        i +=1
        q = b.split("Q"+str(i)+":")[1].split("\nA"+str(i)+":")[0].lstrip()
        ans = b.split("A"+str(i)+":")[1].split("\n\nQ"+str(i)+":")[0].lstrip()
        instruction_data.append({'instruction':q,'output':ans})

In [ ]:
import random
for a in random.sample(instruction_data, 5):
    print(a)

{'instruction': '국선도의 본래 이름은 무엇인가요?', 'output': '국선도의 본래 이름은 밝돌법입니다.'}
{'instruction': '국선도 밝돌법은 몇 년 정도 수련해야 효과를 얻을 수 있을까요?', 'output': '1,2년 정도 수련하면 효과를 얻을 수 있습니다.'}
{'instruction': '자연치유는 어떤 목적으로 수행되는 근본적 수련법인가?', 'output': '자연치유는 몸과 마음의 모든 병폐를 뿌리째 뽑아 낼 수 있는 근본적 수련법이다.'}
{'instruction': '선도법(仙道法) 단계는 어떤 단계인가요?', 'output': '선도법(仙道法) 단계는 천인묘합(妙)의 이상을 관념이 아니라 직접 몸과 마음으로 구현하는 초인적 단계입니다.'}
{'instruction': '정도(正道)는 어떻게 살아가야 올바른 길로 갈 수 있는지 알려주는가?', 'output': '정도(正道)는 하늘사람의 도리를 다하여 올바르게 살 수 있는 길로 살아가라고 알려준다.'}


In [ ]:
len(instruction_data)

350

In [ ]:
# 덜 좋은 답변 만들기

### 덜 좋은 답변 만들기

참고자료를 주지 않고, 오로지 상상에만 맡겨서 데이터를 만들기!

In [ ]:
qa = [a['instruction'] for a in instruction_data]

In [ ]:
qa[0]

'국선도는 어떤 목적으로 실천하는 수련법인가요?'

### 질문을 10개 단위로 쪼개기

In [ ]:
offset=0
qa_set=[]
while True:

    if offset >= len(qa):
        break

    qa_set.append("\n".join(["Q"+str(x+1) + ":" + qa[offset+x] for x in range(10)]))

    offset+=10

In [ ]:
print(qa_set[-1])

Q1:국선도의 기화법은 무엇인가요?
Q2:국선도의 기화법을 일반 수련장에서 배울 수 있나요?
Q3:국선도의 기화법은 어떤 종류가 있나요?
Q4:국선도의 기화법에는 어떤 무기술이 있는지 알 수 있을까요?
Q5:국선도의 기화법에 대한 자세한 사항은 어디에서 알 수 있을까요?
Q6:국선도의 기화법은 얼마나 많은 사람들이 배우고 있을까요?
Q7:국선도의 기화법은 전통적인 무술인가요?
Q8:국선도의 기화법을 배워보고 싶은데 어떤 도장에 가야하나요?
Q9:국선도의 기화법은 어떤 목적을 가지고 있나요?
Q10:국선도의 기화법을 배우면 어떤 장점이 있을까요?


In [ ]:
PROMPT="### 주어진 질문들에 대해서 각각 답변해줘\n\
각 답변은 아래와 같은 형식으로 답변해줘.\n\
\n\
A1:답변\n\
A2:답변\n\
...\n\
A10:답변10\n\n\
### 질문들\n\
{questions}\n"

In [ ]:
print(PROMPT.format_map({'questions':qa_set[0]}))

### 주어진 질문들에 대해서 각각 답변해줘
각 답변은 아래와 같은 형식으로 답변해줘.

A1:답변
A2:답변
...
A10:답변10

### 질문들
Q1:국선도는 어떤 목적으로 실천하는 수련법인가요?
Q2:국선도는 어떤 시기부터 전해져왔나요?
Q3:국선도는 몇 개의 단계와 동작으로 구성되어 있나요?
Q4:국선도는 어떤 도법으로 알려져 있나요?
Q5:국선도는 어떤 명칭으로도 불리고 있나요?
Q6:국선도를 현대 인류 사회에 다시 소개한 사람은 누구인가요?
Q7:국선도의 본래 이름은 무엇인가요?
Q8:'밝돌법'이란 단어는 어떤 의미를 가지고 있나요?
Q9:국선도는 어느 시대부터 전해지기 시작했나요?
Q10:국선도는 누구에게 도움을 줄 수 있는 수련법인가요?



In [ ]:
bad_instruction_data = []
for a in tqdm(qa_set):
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                {'role':"system","content":"You are a helpful assistant."},
                {'role':"user","content":a},
                ],
            temperature=1,
        )
        bad_instruction_data.append(response['choices'][0]['message']['content'])
        time.sleep(1)
    except Exception as e:
        error_data.append(a)
        print(title)
        print(e)

100%|██████████| 35/35 [13:22<00:00, 22.91s/it]


In [ ]:
bad_instruction_data[20]

'A1: 중기단법의 전편은 15개의 동작으로 구성되어 있고, 후편은 14개의 동작으로 구성되어 있습니다.\nA2: 50동작을 한 번에 수련하는 경우는 중기단법 사범단에서 가능합니다. 사범단은 대한태권도협회에서 공인된 사범들이 소속된 단체입니다.\nA3: 중기단법에서 호의 길이와 흡의 길이는 개인별로 다를 수 있지만, 일반적으로 호의 길이는 흡의 길이의 2배 정도로 설정됩니다.\nA4: 중기단법은 기초, 초급, 중급, 고급 단계로 진행됩니다. 각 단계에서는 더 복잡하고 깊이 있는 동작들을 배우며 기술 수준을 향상시킵니다.\nA5: 중기단법에서 호흡은 세 가지 단계로 진행됩니다. 첫째, 질식호흡으로 짧은 시간 동안 호흡을 참습니다. 둘째, 자연호흡으로 자연스럽고 균형있게 호흡합니다. 셋째, 조작호흡으로 호흡을 의도적으로 조절하며, 깊고 효율적인 호흡을 하게 됩니다.\nA6: 중기단법을 수련하다 보면 호흡이 점점 깊어지고 심호흡을 하게 됩니다. 이렇게 되면 몸의 에너지가 더욱 효과적으로 활용되고, 집중력이 높아지며 신체와 정신의 건강을 향상시킬 수 있습니다.\nA7: 중기단법을 시중 수련장에서 완벽하게 수련하는 것은 매우 어렵습니다. 중기단법은 전문적인 지도자의 도움과 개별적인 수련 공간이 필요하기 때문입니다.\nA8: 중기단법을 수련하는 데 걸리는 시간은 개인의 노력 및 수련 빈도에 따라 상이합니다. 그러나 주 1~2회 적절한 수련 시간을 가진다면 몇 달에서 여러 년이 걸릴 수 있습니다.\nA9: 중기단법에서는 주에 맞추어 호의 길이와 흡의 길이를 동일하게 설정하지 않습니다. 중기단법에서는 호의 길이와 흡의 길이를 개별적으로 조절하며 수련합니다.\nA10: 중기단법을 배우고 나면 신체의 근력과 유연성이 향상되고, 평형감각과 명상력이 증가합니다. 또한, 정신을 집중시키고 몸과 마음의 조화를 이루는 기술을 배워 행복하고 건강한 라이프스타일을 유지할 수 있습니다.'

In [ ]:
import json
with open("/content/drive/MyDrive/KoChatGPT/data_kochatgpt/kuksundo_rm.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in instruction_data: file.write(json.dumps(i, ensure_ascii=False) + "\n")

with open("/content/drive/MyDrive/KoChatGPT/data_kochatgpt/kuksundo_rm_bad.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in bad_instruction_data: file.write(json.dumps(i, ensure_ascii=False) + "\n")

In [ ]:
instruction_data=[]
for a in qa_data:
    for i, b in enumerate(a.split("\n\n")):
        i +=1
        q = b.split("Q"+str(i)+":")[1].split("\nA"+str(i)+":")[0].lstrip()
        ans = b.split("A"+str(i)+":")[1].split("\n\nQ"+str(i)+":")[0].lstrip()
        instruction_data.append({'instruction':q,'output':ans})

In [ ]:
qa_set[0]

"Q1:국선도는 어떤 목적을 가지고 있는가?\nQ2:국선도는 어떤 요소와 관계없이 누구나 수련할 수 있는가?\nQ3:국선도는 어떻게 구성되어 있는가?\nQ4:국선도의 명칭인 '밝돌법'은 무엇을 의미하는가?\nQ5:국선도는 어떤 명칭들로도 불린 적이 있는가?\nQ6:국선도는 어떤 시대부터 전해져 내려왔는가?\nQ7:국선도는 어떤 도법으로서 소개되었는가?\nQ8:국선도는 어떤 수련 방법을 의미하는가?\nQ9:국선도의 창시자는 누구인가?\nQ10:국선도는 어떤 시기에 현대 인류 사회에 소개되었는가?"

In [ ]:
print(bad_instruction_data[-1])

A1: 건곤단법에서는 헛발질도 필요합니다. 헛발질은 몸의 균형과 협응력을 향상시키고 신체의 기동성을 향상시킬 수 있기 때문입니다.
A2: 건곤단법을 수행하기 위해 도장에서는 다음과 같은 조건을 요구합니다. 첫째, 전문 지도자의 안내가 필요하며, 규칙과 원칙을 준수해야 합니다. 둘째, 안전한 환경과 적절한 장비가 필요합니다. 셋째, 꾸준한 훈련과 노력이 필요하며, 목표 설정과 집중력을 유지해야 합니다.
A3: 건곤단법은 동작, 호흡, 지식, 발음 등을 포함한 종합적인 무예 교육 체계입니다. 몸과 마음을 통합하고 신체적, 정신적인 발전을 도모하는 것이 주된 목표입니다.
A4: 건곤단법에서는 다음과 같은 순서로 동작을 수행합니다. 먼저, 선천적 연습을 통해 근육을 활성화시키고 몸의 기동성을 향상시킵니다. 그 다음, 호흡을 조절하여 에너지를 충전하고 몸의 균형을 유지합니다. 마지막으로, 지식과 방어 기술을 통해 스스로를 보호하고 상대방과의 접촉에서 이기기 위해 훈련합니다.
A5: 건곤단법에서 호흡과 지식의 길이는 훈련자의 능력과 경험에 따라 다를 수 있습니다. 초기 단계에서는 호흡과 지식이 짧을 수 있지만, 꾸준한 훈련을 통해 점차 길어질 수 있습니다.
A6: 건곤단법에서는 지식이 일정 수준 이상으로 길어지면 "기가"라는 순기능이 작동하기 시작합니다. 기가는 몸의 에너지를 증폭시켜 강력한 공격과 방어를 가능하게 합니다.
A7: 건곤단법에서는 곤의 신법인 임독유통은 다른 무예 기술과 비슷한 방식으로 이루어집니다. 이는 상대방의 공격을 막거나 상대방을 제압하기 위해 다양한 기술과 행동을 사용하는 것을 의미합니다.
A8: 건곤단법을 수행하기 위해서는 개인의 능력과 노력에 따라 필요한 시간이 달라집니다. 꾸준하고 집중된 훈련을 통해 발전이 이루어지며, 시간이 지남에 따라 실력이 향상됩니다.
A9: 건곤단법에서 뺑이를 많이 치려면 다음과 같은 조건이 필요합니다. 첫째, 꾸준한 훈련과 노력이 필요하며, 반복적인 연습을 통해 기술을 익히고 습관화해야 합니다. 둘째, 목표 설정과 

In [ ]:
bad_instruction_data[20]

'Q1: 중기단법의 전편과 후편은 각각 25동작으로 구성되어 있습니다.\nQ2: 50동작을 한 번에 수련하는 경우, 중기단법 실습장에서 가능합니다.\nQ3: 중기단법에서 호의 길이는 2/7로, 흡의 길이는 5/7로 설정됩니다.\nQ4: 중기단법을 수련하면 선단, 중단, 후단, 전단, 중전단, 중후단, 후전단, 전후단, 중전후단의 단계로 진행됩니다.\nQ5: 중기단법에서 호흡은 호기와 흡기로 구성되며, 호기는 코를 통해 억제되고, 흡기는 입으로 이루어집니다.\nQ6: 중기단법에서 호흡이 점점 깊어지면 체내의 에너지 흐름과 조화로워지며, 심신의 통일과 안정을 느끼게 됩니다.\nQ7: 중기단법은 시중 수련장에서는 몇 분 내에 마치는 것은 어렵습니다. 전문가의 지도 아래에서 연습하는 것이 필요합니다.\nQ8: 중기단법을 수련하는 데 걸리는 시간은 개인에 따라 다르며, 노력과 꾸준한 연습을 통해 숙달되기까지 오랜 시간이 소요될 수 있습니다.\nQ9: 중기단법에서는 특정 주에 맞추어 호의 길이와 흡의 길이를 동일하게 설정하지 않습니다. 호의 길이와 흡의 길이는 비율에 따라 정해집니다.\nQ10: 중기단법을 배우고 나면 몸과 마음의 조화, 에너지 흐름의 통일, 종합적인 건강과 안녕함을 느끼게 됩니다. 또한, 높은 집중력과 내면의 평화를 경험할 수 있습니다.'

In [ ]:
reward_dataset=[]

for question, answer in zip(qa_set, bad_instruction_data):

    for i in range(10):
        i += 1

        if i <= 9:
            q = question.split("Q"+str(i)+":")[1].split("Q"+str(i+1)+":")[0].lstrip().strip()
            a = answer.split("A"+str(i)+":")[1].split("A"+str(i+1)+":")[0].lstrip().strip()
        elif i == 10 :
            q = question.split("Q"+str(i)+":")[1].lstrip().strip()
            a = answer.split("A"+str(i)+":")[1].lstrip().strip()

        reward_dataset.append({'prompt':q,'rejected':a})


In [ ]:
len(reward_dataset)

350

In [ ]:
for i, a in enumerate(reward_dataset):
    a['chosen'] = instruction_data[i]['output']

In [ ]:
reward_dataset

[{'prompt': '국선도는 어떤 목적으로 실천하는 수련법인가요?',
  'rejected': '국선도는 주로 사람들의 신체와 정신 건강을 향상시키기 위해 수련하는 것을 목적으로 합니다.',
  'chosen': '국선도는 건강한 몸, 마음과 정신을 가진 전인적인 인간으로 거듭나기 위한 자기계발 수련법입니다.'},
 {'prompt': '국선도는 어떤 시기부터 전해져왔나요?',
  'rejected': '국선도는 신라 시대부터 전해져 왔으며, 고려와 조선 시대에도 계속해서 전승되어 왔습니다.',
  'chosen': '국선도는 약 9,800년 전부터 전해져왔습니다.'},
 {'prompt': '국선도는 몇 개의 단계와 동작으로 구성되어 있나요?',
  'rejected': '국선도는 주로 7개의 단계와 다양한 동작으로 구성되어 있습니다.',
  'chosen': '국선도는 9단법 37단계 430여 개 동작으로 구성되어 있습니다.'},
 {'prompt': '국선도는 어떤 도법으로 알려져 있나요?',
  'rejected': '국선도는 주로 기본적인 군용 기술인 무예로 알려져 있습니다.',
  'chosen': '국선도는 점진적이고 체계적인 단전행공(丹田行功)의 도법(道法)으로 알려져 있습니다.'},
 {'prompt': '국선도는 어떤 명칭으로도 불리고 있나요?',
  'rejected': '국선도는 "국권 수련법"이라는 명칭으로도 불리고 있습니다.',
  'chosen': '국선도는 고조선의 소도, 고구려의 조의선인제도, 신라의 화랑도 등으로도 불리고 있습니다.'},
 {'prompt': '국선도를 현대 인류 사회에 다시 소개한 사람은 누구인가요?',
  'rejected': '국선도를 현대 인류 사회에 다시 소개한 사람은 오일장 선생입니다.',
  'chosen': '국선도를 현대 인류 사회에 다시 소개한 사람은 청산(본명 고한영) 스승인 청운도인입니다.'},
 {'prompt': '국선도의 본래 이름은 무엇인가요?',
  'rejected': '국선도

In [ ]:
len(reward_dataset)

350

In [ ]:
train_rm=[]
test_rm=[]
for a in reward_dataset:
    if random.random()< 0.1:
        test_rm.append(a)
    else:
        train_rm.append(a)

In [ ]:
import json
with open("/content/drive/MyDrive/KoChatGPT/data_kochatgpt/kuksundo_train_rm.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in train_rm: file.write(json.dumps(i, ensure_ascii=False) + "\n")

with open("/content/drive/MyDrive/KoChatGPT/data_kochatgpt/kuksundo_test_rm.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in test_rm: file.write(json.dumps(i, ensure_ascii=False) + "\n")

In [ ]:
#

# Stage 3 데이터 만들기, 질문 Only

In [ ]:
data[0]

{'instruction': '다음 주제에 대하여, 위키피디아에 나와 있는 그대로 말해주세요.\n주제:국선도의 개요1\n',
 'output': '국선도(밝돌법)는 9,800년의 역사와 전통을 가진 것으로 알려진 한국 고유의 심신 수련법으로, 남녀노소, 인종, 종교, 질병 유무와 무관하게 누구든지 심신 수련을 통해 건강한 몸, 마음과 정신을 가진 전인적인 인간으로 거듭날 수 있도록 돕는 자기계발 수련법이다.    9단법 37단계 430여 개 동작으로 구성된 점진적이고 체계적인 단전행공(丹田行功)의 도법(道法)으로 한국 고유의 명상과학 프로그램이다.    심신수련과 인재 양성의 도법으로 약 9,800년 전부터 전해져 내려와 고조선의 소도, 고구려의 조의선인제도, 신라의 화랑도 등으로 불렸고, 고려 이후에는 산중 도법으로 전해 내려오다가 산중 수도인 청산(靑山, 본명 고한영)이 스승 청운도인의 "모두의 유익을 위해 도법을 전하라"는 명을 받고 1967년에 사회로 나와 국선도(밝돌법)를 현대 인류 사회에 다시 소개하였다.    밝돌법은 국선도의 본래 이름으로, ‘밝’은 생명의 에너지(태양, 밝음), ‘돌’은 순환하는(돌고 도는) 우주 자연의 법칙을 뜻하며, ‘밝돌법’은 우주 질서의 순리에 맞게 생명을 양생(養生)하는 수련방법을 의미한다.    국선도는 상고시대부터 전해오는 한국의 고유한 전통 심신수련법이다.'}

In [ ]:
from tqdm import tqdm

In [ ]:
qa_data=[]
error_data=[]

In [ ]:
PROMPT="다음 주제와 내용을 보고 답변 할 수 있는 다양한 형태와 난이도의 창의적인 질문 20개를 만들어줘. n주제:{title}\n내용:{content}\n\
질문과 질문 사이는 반드시 ###을 붙여줘.\n\
답변의 형태는 다음과 같아.\n\
\n\
### Q1:질문1\n\
### Q2:질문2\n\
### Q3:질문3\n\
...\n\
### Q20:질문20\n\
"

for a in tqdm(data):

    title = a['instruction'].split("주제:")[-1]
    content = a['output']
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=[
                {'role':"system","content":"You are a helpful assistant."},
                {'role':"user","content":PROMPT.format_map({'title':title, 'content':content})},
                ],
            temperature=1
        )
        qa_data.append(response['choices'][0]['message']['content'])
    except Exception as e:
        error_data.append(a)
        print(title)
        print(e)

100%|██████████| 35/35 [26:48<00:00, 45.96s/it]


In [ ]:
qa_data[-1]

'### Q1: 국선도의 기원은 어디에 있을까요?\n### Q2: 국선도를 연마하기 위해 필요한 신체적인 능력은 무엇이 있을까요?\n### Q3: 국선도에서 사용하는 무기술은 어떤 종류가 있을까요?\n### Q4: 국선도의 주요 기술 중 어떤 기술이 가장 어려운 것으로 알려져 있나요?\n### Q5: 국선도를 평소에 수련하는 연습 방법은 어떤 것이 있을까요?\n### Q6: 국선도의 훈련 과정은 어떻게 진행되나요?\n### Q7: 국선도를 배우기 위해선 어떤 기초 지식이 필요한가요?\n### Q8: 국선도에서 가장 중요하게 여기는 가치는 무엇인가요?\n### Q9: 국선도 수련을 통해 어떤 점이 개선될 수 있을까요?\n### Q10: 국선도에서의 경기 규칙은 어떻게 되나요?\n### Q11: 국선도의 발전을 위해선 어떤 노력이 필요한가요?\n### Q12: 국선도를 수련하는 사람들은 보통 어떤 혜택을 누릴 수 있을까요?\n### Q13: 국선도의 전통적인 의미와 현대적인 측면에서 어떤 차이점이 있나요?\n### Q14: 국선도를 배우는 데에는 어떤 장점이 있을까요?\n### Q15: 국선도를 수련하면 어떻게 자기개발에 도움이 될 수 있나요?\n### Q16: 국선도의 특징 중 가장 독특한 것은 무엇인가요?\n### Q17: 국선도를 배우는 것과 실제로 적용하는 것 사이에 어떤 차이가 있을까요?\n### Q18: 국선도에서 가장 중요한 요소는 무엇인가요?\n### Q19: 국선도 수련을 통해 어떻게 자신감을 키울 수 있을까요?\n### Q20: 국선도를 수련하는데 필요한 장비와 도구는 어떤 것들이 있을까요?'

In [ ]:
prompt_dataset=[]

for question in qa_data:

    try:
        for i in range(20):
            i += 1

            if i <= 19:
                q = question.split("### Q"+str(i)+":")[1].split("### Q"+str(i+1)+":")[0].lstrip().strip()
            elif i == 20 :
                q = question.split("### Q"+str(i)+":")[1].lstrip().strip()

            prompt_dataset.append({'prompt':q})
    except:
        print(question)
        break


In [ ]:
prompt_dataset[-1]

{'prompt': '국선도를 수련하는데 필요한 장비와 도구는 어떤 것들이 있을까요?'}

In [ ]:
len(prompt_dataset)

700

In [ ]:
import random
train=[]
test=[]
for a in prompt_dataset:
    if random.random() < 0.1:
        test.append(a)
    else:
        train.append(a)

In [ ]:
import json
with open("/content/drive/MyDrive/KoChatGPT/data_kochatgpt/kuksundo_train_ppo.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in train: file.write(json.dumps(i, ensure_ascii=False) + "\n")

with open("/content/drive/MyDrive/KoChatGPT/data_kochatgpt/kuksundo_test_ppo.jsonl" , encoding= "utf-8",mode="w") as file:
	for i in test: file.write(json.dumps(i, ensure_ascii=False) + "\n")